In [1]:
import geopandas as gpd
import pandas as pd
import shapely.geometry as sgeo

In [2]:
if "snakemake" in locals():
    demand_path = snakemake.input["demand"]
    areas_path = snakemake.input["areas"]
    output_path = snakemake.output[0]
    area = snakemake.params["area"]

else:
    demand_path = "../../../demand/results/paris/trips/passenger_trips_seed1000.gpkg"
    areas_path = "../../resources/paris/spatial.gpkg"
    output_path = "../../results/paris/demand/filtered.gpkg"
    area = "main"

In [3]:
# Load area
df_areas = gpd.read_file(areas_path)
df_areas = df_areas[df_areas["name"] == area]
assert len(df_areas) == 1

In [4]:
# Load demand
df_demand = gpd.read_file(demand_path)

df_demand["origin_inside"] = False
df_demand["destination_inside"] = False

In [5]:
# Intersect origins and destinations
df_origin = df_demand[["geometry"]].copy()
df_origin["geometry"] = df_origin["geometry"].apply(lambda line: sgeo.Point(*line.coords[0]))
df_origin = gpd.sjoin(df_origin, df_areas, predicate = "within")
df_demand.loc[df_origin.index, "origin_inside"] = True

df_destination = df_demand[["geometry"]].copy()
df_destination["geometry"] = df_destination["geometry"].apply(lambda line: sgeo.Point(*line.coords[1]))
df_destination = gpd.sjoin(df_destination, df_areas, predicate = "within")
df_demand.loc[df_destination.index, "destination_inside"] = True

In [6]:
# Filter relevant trips
df_demand = df_demand[
    (df_demand["is_access"] & df_demand["origin_inside"]) |
    (~df_demand["is_access"] & df_demand["destination_inside"])
]

In [7]:
# Output
df_demand.to_file(output_path)